In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
dagnelies_deepfake_faces_path = kagglehub.dataset_download('dagnelies/deepfake-faces')

print('Data source import complete.')


In [ ]:
!pip install torchviz

In [ ]:
# -----------------------------------------------------------------------------
# 1. Imports et Configuration
# -----------------------------------------------------------------------------

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import ViTForImageClassification
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, matthews_corrcoef
from sklearn.metrics import f1_score, cohen_kappa_score, log_loss, recall_score, precision_score
from sklearn.metrics import roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import numpy as np
import os
import cv2
import pandas as pd
from pathlib import Path
import time
import warnings
warnings.filterwarnings('ignore')

# -----------------------------------------------------------------------------
# 2. Classes et Fonctions de Base
# -----------------------------------------------------------------------------

# Dataset personnalisé pour les images de deepfake
class DeepfakeDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        # Conversion BGR -> RGB car OpenCV charge en BGR mais PyTorch attend RGB
        self.images = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in images]
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Transformations de données
def get_transforms():
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

    return transform

# Charger le modèle ViT pré-entraîné
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=2,  # 2 classes : fake et real
    ignore_mismatched_sizes=True  # Ignorer les incompatibilités de taille
)

# Charger le feature extractor (pour le prétraitement des images)
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")# -----------------------------------------------------------------------------
# 3. Chargement des Données
# -----------------------------------------------------------------------------

# Créer les dataloaders
transform = get_transforms()
train_dataset = DeepfakeDataset(X_train, y_train, transform=transform)
val_dataset = DeepfakeDataset(X_val, y_val, transform=transform)
test_dataset = DeepfakeDataset(X_test, y_test, transform=transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# -----------------------------------------------------------------------------
# 4. Entraînement du Modèle
# -----------------------------------------------------------------------------

# Configurer l'utilisation du GPU/CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Instancier le modèle
#model = CustomViTWithLatentAttention(num_classes=2, num_latents=64)
model = model.to(device)

# Optimizer et critère de perte
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-4, weight_decay=0.01)
criterion = CrossEntropyLoss()
scaler = torch.amp.GradScaler()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1)

# Dossier pour sauvegarder les résultats
results_dir = f"deepfake_results_{time.strftime('%Y%m%d_%H%M%S')}"
os.makedirs(results_dir, exist_ok=True)

# Boucle d'entraînement
epochs = 15
patience = 5
best_valid_loss = float('inf')
patience_counter = 0

for epoch in range(epochs):
    epoch_start_time = time.time()
    model.train()
    running_loss, running_accuracy = 0.0, 0.0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast(device_type='cuda' if device == 'cuda' else 'cpu'):
            outputs = model(images)  # outputs est un objet ImageClassifierOutput
            logits = outputs.logits  # Extraire les logits
            loss = criterion(logits, labels)  # Utiliser les logits pour calculer la perte

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        preds = torch.argmax(logits, dim=1)  # Utiliser les logits pour les prédictions
        running_accuracy += accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = running_accuracy / len(train_loader)

    # Validation
    model.eval()
    valid_loss, valid_accuracy = 0.0, 0.0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Valid]"):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)  # outputs est un objet ImageClassifierOutput
            logits = outputs.logits  # Extraire les logits
            loss = criterion(logits, labels)  # Utiliser les logits pour calculer la perte

            valid_loss += loss.item()
            preds = torch.argmax(logits, dim=1)  # Utiliser les logits pour les prédictions
            valid_accuracy += accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())

    valid_loss = valid_loss / len(val_loader)
    valid_accuracy = valid_accuracy / len(val_loader)

    # Early stopping
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        patience_counter = 0
        torch.save(model.state_dict(), os.path.join(results_dir, 'best_model.pth'))
        print(f"Model saved with improved validation loss: {valid_loss:.4f}")
    else:
        patience_counter += 1

    scheduler.step(valid_loss)
    epoch_time = time.time() - epoch_start_time

    print(f"Epoch {epoch+1}/{epochs} - Time: {epoch_time:.2f}s")
    print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}")
    print(f"Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")
    print("-" * 50)

    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs")
        break


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: cuda


Epoch 1/15 [Valid]: 100%|██████████| 245/245 [01:33<00:00,  2.61it/s]


Model saved with improved validation loss: 0.5828
Epoch 1/15 - Time: 741.28s
Train Loss: 0.6628, Train Accuracy: 0.6041
Valid Loss: 0.5828, Valid Accuracy: 0.6806
--------------------------------------------------


Epoch 2/15 [Valid]: 100%|██████████| 245/245 [01:34<00:00,  2.60it/s]


...

Epoch 7/15 - Time: 741.66s
Train Loss: 0.1028, Train Accuracy: 0.9616
Valid Loss: 0.6425, Valid Accuracy: 0.8540
--------------------------------------------------


Epoch 8/15 [Valid]: 100%|██████████| 245/245 [01:34<00:00,  2.60it/s]


Epoch 8/15 - Time: 741.36s
Train Loss: 0.0823, Train Accuracy: 0.9798
Valid Loss: 1.0932, Valid Accuracy: 0.8676
--------------------------------------------------


Epoch 9/15 [Valid]: 100%|██████████| 245/245 [01:34<00:00,  2.60it/s]

Epoch 9/15 - Time: 743.45s
Train Loss: 0.0337, Train Accuracy: 0.9931
Valid Loss: 1.3341, Valid Accuracy: 0.8846
--------------------------------------------------
Early stopping triggered after 9 epochs


In [ ]:
# -----------------------------------------------------------------------------
# 5. Évaluation du Modèle
# -----------------------------------------------------------------------------

# Charger le meilleur modèle
model.load_state_dict(torch.load(os.path.join(results_dir, 'best_model.pth')))

# Évaluation sur l'ensemble de test
model.eval()
all_preds = []
all_labels = []
all_probs = []
test_loss = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)  # outputs est un objet ImageClassifierOutput
        logits = outputs.logits  # Extraire les logits
        loss = criterion(logits, labels)  # Utiliser les logits pour calculer la perte
        test_loss += loss.item()

        probs = torch.softmax(logits, dim=1)  # Calculer les probabilités à partir des logits
        _, preds = torch.max(probs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

# Convertir all_probs en un tableau NumPy 2D
all_probs_np = np.array(all_probs)

# Calcul des métriques
test_loss = test_loss / len(test_loader)
accuracy = accuracy_score(all_labels, all_preds)
roc_auc_value = roc_auc_score(all_labels, all_probs_np[:, 1])
mcc = matthews_corrcoef(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
kappa = cohen_kappa_score(all_labels, all_preds)
logloss = log_loss(all_labels, all_probs)
recall = recall_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)

# Matrice de confusion
cm = confusion_matrix(all_labels, all_preds)
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)

# Affichage des métriques
print("\nTest Metrics:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC-AUC: {roc_auc_value:.4f}")
print(f"MCC: {mcc:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Kappa: {kappa:.4f}")
print(f"Log Loss: {logloss:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")

# Sauvegarder les métriques
metrics_df = pd.DataFrame({
    'test_loss': [test_loss],
    'accuracy': [accuracy],
    'roc_auc': [roc_auc_value],
    'mcc': [mcc],
    'f1': [f1],
    'specificity': [specificity],
    'kappa': [kappa],
    'logloss': [logloss],
    'recall': [recall],
    'precision': [precision]
})
metrics_df.to_csv(os.path.join(results_dir, 'test_metrics.csv'), index=False)


Evaluating: 100%|██████████| 204/204 [01:19<00:00,  2.58it/s]


Test Metrics:
Test Loss: 0.3443
Accuracy: 0.8635
ROC-AUC: 0.9432
MCC: 0.7295
F1 Score: 0.8690
Specificity: 0.8211
Kappa: 0.7269
Log Loss: 0.3446
Recall: 0.9058
Precision: 0.8351


In [ ]:
# Convertir all_probs en un tableau NumPy 2D
all_probs_np = np.array(all_probs)

# Calculer ROC-AUC
roc_auc_value = roc_auc_score(all_labels, all_probs_np[:, 1])  # Accéder à la deuxième colonne
print(f"ROC-AUC: {roc_auc_value:.4f}")

ROC-AUC: 0.9432
